In [ ]:
import skfuzzy as fuzz
def fuzzy_C(image,n_classes):
    # Flatten the image array and normalize
    data = image.flatten()
    data_norm = data / 255.0  # Normalize to [0, 1]

    # Reshape data for clustering
    data_reshaped = data_norm.reshape((1, -1))

    # Specify the number of clusters
    clusters = n_classes

    # Apply Fuzzy C-means
    cntr, u, u0, d, jm, p, fpc = fuzz.cluster.cmeans(
        data_reshaped, clusters, 2, error=0.005, maxiter=1000, init=None)

    # Find the cluster membership for each pixel
    cluster_membership = np.argmax(u, axis=0)

    # Map back to image shape
    segmented_image = cluster_membership.reshape(image.shape)

    # Convert the segmented image into uint8 (for displaying)
    segmented_image = np.uint8(255 * segmented_image / clusters)
    return segmented_image

In [ ]:
import cv2
import numpy as np
from imutils.perspective import four_point_transform 

# cap = cv2.VideoCapture(1)
# # +cv2.CAP_DSHOW
# if not cap.isOpened():
#     print("Cannot open camera")
#     exit()

WIDTH, HEIGHT = 600, 300
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)


def scan_detection_1(image_,bl_sz,const):
    global document_contour
    image=image_
    im_cop=image_
    # khởi tạo contour đầu tiên, bằng kích thước của frame
    document_contour= np.array([[0,0],[WIDTH,0],[WIDTH,HEIGHT],[0,HEIGHT]])

    # xử lý ảnh cơ bản, chuyển màu gray, blur ảnh để chống nhiễu và giảm độ chi tiết 
    gray=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur= cv2.GaussianBlur(gray, (15,15),0)
    # blur= cv2.GaussianBlur(blur, (5,5),0)
    # blur= cv2.GaussianBlur(blur, (5,5),0)
    # blur= cv2.GaussianBlur(blur, (5,5),0)
    frame_gray= cv2.adaptiveThreshold(
    blur,                # source image
    255,                       # max value to use with the THRESH_BINARY
    cv2.ADAPTIVE_THRESH_GAUSSIAN_C, # adaptive method, Gaussian weighting
    cv2.THRESH_BINARY,         # threshold type
    bl_sz,                        # block size: size of a pixel neighborhood used to calculate threshold value
    const                       # constant subtracted from the mean or weighted sum
    )
    # frame_gray = cv2.Canny(blur, 100, 300)

    dst = cv2.cornerHarris(frame_gray, blockSize=2, ksize=3, k=0.04)
    # Result is dilated for marking the corners
    dst = cv2.dilate(dst, None)
    # Threshold for an optimal value, marking the corners in red
    im_cop[dst > 0.1 * dst.max()] = [0, 0, 255]
    contours,_ = cv2.findContours(frame_gray, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours=sorted(contours, key=cv2.contourArea,reverse=True)
    max_area= 500
    for contour in contours:
        area=cv2.contourArea(contour)
        print(area)
        if area > 100:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.15*peri, True)
            print(len(approx))
            if  area > max_area:#and len(approx)==4
                print(f"accepted area {area}")
                document_contour=approx
                max_area=area
    cv2.drawContours(image,[document_contour],-1,(0,255,0),3)
    return image,blur,frame_gray,contours,im_cop,bl_sz,const

imagea=cv2.imread("/Users/mac/Dev/Computer_Vision/Image 2.jpeg")
if imagea is None:
    print("Failed to load image")
    exit()
for bll in range(3,25,2):
    for c in range(3,25,2):
        im,bl,gr,con,corner,bl,cons=scan_detection_1(imagea,bll,c)
        cv2.imwrite(f"/Users/mac/Dev/Computer_Vision/outputs/corner/image_{bl}_{cons}.jpg",im)
# cv2.imshow("m", im)
# cv2.imshow("blurred",bl)
# cv2.imshow("gray",gr)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Can't receive frame (stream end?). Exiting ...")
#         break
#     # frame = cv2.rotate(frame, cv2.ROTATE_180)
    
#     frame_copy = frame.copy()
#     _1,_2,_3=scan_detection_1(frame_copy)
#     # c=fuzzy_C(frame_copy,5)
#     cv2.imshow("custom_1",_1)
#     if cv2.waitKey(1) & 0xFF == 27:
#         break

# cap.release()
# cv2.destroyAllWindows()


In [ ]:
from imutils.perspective import four_point_transform 

In [1]:
import cv2
import numpy as np
from imutils.perspective import four_point_transform
import pytesseract

cap = cv2.VideoCapture(1)

WIDTH, HEIGHT = 800, 600
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)


def scan_detection(image):
    global document_contour

    document_contour = np.array([[0, 0], [WIDTH, 0], [WIDTH, HEIGHT], [0, HEIGHT]])

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    dst = cv2.cornerHarris(gray, blockSize=3, ksize=5, k=0.07)
    # Result is dilated for marking the corners
    dst = cv2.dilate(dst, None)
    # Threshold for an optimal value, marking the corners in red
    frame[dst > 0.1 * dst.max()] = [0, 0, 255]

    _, threshold = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    contours, _ = cv2.findContours(threshold, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    max_area = 500000
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > 400000:
            peri = cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, 0.015 * peri, True)
            if area > max_area and len(approx) == 4:
                document_contour = approx
                max_area = area

    cv2.drawContours(frame, [document_contour], -1, (0, 255, 0), 3)
    return threshold 


while True:

    _, frame = cap.read()
    frame_copy = frame.copy()

    thrh=scan_detection(frame_copy)

    cv2.imshow("input", frame)
    cv2.imshow("ouput", thrh)

    # warped = four_point_transform(frame_copy, document_contour.reshape(4, 2))
    # cv2.imshow("Warped", warped)

    # print(ocr_text)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()

2024-04-24 15:43:09.964 Python[49698:11941627] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
2024-04-24 15:43:12.012 Python[49698:11941627] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.
